In [1]:
import yaml

def write_pocket2mol_config(num_samples: int, out_path: str = "sample_for_pdb.yml"):
    """
    num_samples 값을 받아 Pocket2Mol용 yml 설정 파일을 생성한다.
    out_path : 저장할 yml 경로 (기본: sample_for_pdb.yml)
    """
    cfg = {
        "model": {
            "checkpoint": "./ckpt/pretrained_Pocket2Mol.pt"
        },
        "sample": {
            "seed": 2020,
            "num_samples": int(num_samples),
            "beam_size": 300,
            "max_steps": 50,
            "threshold": {
                "focal_threshold": 0.5,
                "pos_threshold": 0.25,
                "element_threshold": 0.3,
                "hasatom_threshold": 0.6,
                "bond_threshold": 0.4,
            },
        },
    }

    with open(out_path, "w") as f:
        # 보기 좋게 쓰고 싶으면 default_flow_style=False
        yaml.dump(cfg, f, sort_keys=False)

    print(f"Config written to: {out_path}")


In [2]:
out_path = "/opt/git_tools/Pocket2Mol_LNPAI/configs/sample_for_pdb2.yml"
write_pocket2mol_config(20, )

Config written to: sample_for_pdb.yml


In [1]:
import subprocess

cmd = [
    "docker", "run", "--rm", "-it",
    "--gpus", "all",
    "--ipc=host",
    "--volume", "//opt/git_tools/Pocket2Mol_LNPAI/_test/test:/root/result",
    "--volume", "/opt/git_tools/Pocket2Mol_LNPAI/ckpt:/opt/Pocket2Mol/ckpt",
    "pocket2mol:cu113",
    "taskset", "-c", "0",
    "python", "sample_for_pdb.py",
    "--pdb_path", "/root/result/4yhj.pdb",
    "--center", " 32.0,28.0,36.0",
    "--outdir", "/root/result",
    "--config", "/root/result/test.yml"
]

# 실행
result = subprocess.run(cmd, text=True, capture_output=True)

print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)

STDOUT:
 
STDERR:
 the input device is not a TTY



In [ ]:
import subprocess

cmd = [
    "docker", "run", "--rm", "-it",
    "--gpus", "all",
    "--ipc=host",
    "--volume", "/opt/git_tools/Pocket2Mol_LNPAI/_test/input:/root/p2m_input",
    "--volume", "/opt/git_tools/Pocket2Mol_LNPAI/_test/output:/root/p2m_output",
    "--volume", "/opt/git_tools/Pocket2Mol_LNPAI/ckpt:/opt/Pocket2Mol/ckpt",
    "pocket2mol:cu113",
    "taskset", "-c", "0",
    "python", "sample_for_pdb.py",
    "--pdb_path", "/opt/Pocket2Mol/example/4yhj.pdb",
    "--center", " 32.0,28.0,36.0",
    "--outdir", "/root/p2m_output"
]

# 실행
result = subprocess.run(cmd, text=True, capture_output=True)

print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)

In [18]:
from pathlib import Path
import numpy as np

# 자주 등장하는 소형 용매/첨가제/버퍼/폴리올/리간드 아님(확장 가능)
SKIP_RESNAMES_BASE = {
    # 물
    "HOH", "WAT", "DOD",
    # 할라이드/작은 이온성 음이온
    "CL", "BR", "IOD", "I", "F",
    # 황산/인산/포메이트/아세트산 등 소형 음이온
    "SO4", "PO4", "FMT", "ACT", "NO3", "SCN", "CO3",
    # 금속/양이온 (resname=원소기호 형태)
    "LI", "NA", "K", "MG", "CA", "MN", "ZN", "FE", "CO", "NI", "CU", "CD", "HG",
    "SR", "CS", "AL", "GA", "CR", "V", "YB", "PT", "AU", "AG", "TI",
    # 폴리올/크리스탈라이제이션 보조물
    "GOL", "EDO", "PG4", "PEG", "PEO", "MPD", "BME",
    # 일반적 버퍼/가교제
    "TRS", "TES", "MES", "MOPS", "HEP", "EPE", "TES", "TMA", "IPA", "EOH", "ETH",
    # 기타 자주 보이는 작은 것들
    "ACE", "NAG", "NDG", "MAN", "GAL", "GLC", "BMA", "SUC", "MAL",
}

# 금속 원소(원자 element 컬럼 기준) – 원소로 판별할 때 사용
METAL_ELEMENTS = {
    "LI","NA","K","MG","CA","MN","ZN","FE","CO","NI","CU","CD","HG",
    "SR","CS","AL","GA","CR","V","YB","PT","AU","AG","TI","RU","IR","OS","PB","BA","RB"
}

def _is_hydrogen(atom_name: str, element_field: str) -> bool:
    el = (element_field or "").strip().upper()
    if el:
        return el in {"H", "D", "T"}
    return atom_name.strip().upper().startswith("H")

def _is_water_or_small(resname: str) -> bool:
    return resname.upper() in SKIP_RESNAMES_BASE

def _is_metal_resname(resname: str) -> bool:
    return resname.upper() in METAL_ELEMENTS

def _is_metal_element(element_field: str) -> bool:
    return (element_field or "").strip().upper() in METAL_ELEMENTS

def extract_ligands_only(
    pdb_path: str,
    *,
    min_heavy_atoms: int = 8,
    extra_skip: set = None,
):
    """
    작은 이온/용매/금속 등을 제외하고 '리간드'로 보이는 HETATM 잔기만 추출.

    Returns:
        ligands: 리스트[{
            'key': (chain, resname, resseq, icode),
            'coords': np.ndarray[N,3],           # heavy-atom 좌표
            'n_heavy': int,                      # heavy atom 수
            'centroid': np.ndarray[3],           # 무게중심
        }]
        center_str: 가장 큰 리간드의 Pocket2Mol center 문자열
                    (첫 값이 음수면 앞 공백 1칸)
    """
    skip_resnames = set(SKIP_RESNAMES_BASE)
    if extra_skip:
        skip_resnames |= {s.upper() for s in extra_skip}

    residues = {}  # key -> list of (x,y,z)

    with open(pdb_path, "r") as f:
        for line in f:
            if not line.startswith("HETATM"):
                continue

            # PDB 고정 폭 파싱
            resname = line[17:20].strip().upper()
            chain = line[21].strip() or ""
            resseq = line[22:26].strip()
            icode  = line[26].strip() or ""
            atom_name = line[12:16]
            element   = line[76:78] if len(line) >= 78 else ""

            # 수소 제외
            if _is_hydrogen(atom_name, element):
                continue

            # 물/소형 용매/버퍼/할라이드/금속 'resname'로 제외
            if _is_water_or_small(resname) or _is_metal_resname(resname):
                continue

            # 금속 원소(원자 element)로도 제외
            if _is_metal_element(element):
                continue

            # 좌표 파싱
            try:
                x = float(line[30:38]); y = float(line[38:46]); z = float(line[46:54])
            except ValueError:
                continue

            key = (chain, resname, resseq, icode)
            residues.setdefault(key, []).append((x, y, z))

    # 요약 생성 + 최소 원자 수 필터
    ligands = []
    for key, coords in residues.items():
        arr = np.array(coords, dtype=float)
        n_heavy = int(arr.shape[0])
        if n_heavy < min_heavy_atoms:
            continue
        centroid = arr.mean(axis=0)
        ligands.append({
            "key": key,
            # "coords": arr,
            "n_heavy": n_heavy,
            "centroid": np.round(centroid, 2),
        })

    return ligands

def centerStrTranlation(ligands):
    # 리간드로 center 문자열 생성
    center_str = []
    if ligands:
        ligands.sort(key=lambda d: d["n_heavy"], reverse=True)
        cx, cy, cz = ligands[0]["centroid"]
        c0 = f"{cx:.2f}"
        if c0.startswith("-"):
            c0 = " " + c0  # Pocket2Mol 규칙: 첫 값 음수면 앞 공백 1칸
        center_str = f"{c0},{cy:.2f},{cz:.2f}"
    return center_str
    


In [19]:
# 소형 이온/용매/금속 제거 + 최소 heavy atom 8개 이상만 리간드로 인정
ligands = extract_ligands_only(
    "/opt/git_tools/Pocket2Mol_LNPAI/example/3AWG.pdb",
    min_heavy_atoms=8,
)
print("발견된 리간드 개수:", len(ligands))
print("추천 center:", centerStrTranlation(ligands))  # 예: " -12.321,40.577,-30.394"

# 리간드별 좌표 접근
for lig in ligands:
    (chain, resname, resseq, icode) = lig["key"]
    print(f"{resname} {chain}{resseq}{icode}: heavy={lig['n_heavy']}, centroid={lig['centroid']}")
    # coords = lig["coords"]  # (N,3) numpy 배열


발견된 리간드 개수: 0
추천 center: []


In [7]:
ligands

[{'key': ('A', 'AN2', '601', ''),
  'n_heavy': 27,
  'centroid': array([32.10437037, 28.08374074, 36.06892593])},
 {'key': ('B', 'AN2', '601', ''),
  'n_heavy': 27,
  'centroid': array([18.05818519, 31.48311111, 76.03214815])}]